In [4]:
import matplotlib as mpl
mpl.__version__

'3.5.1'

In [5]:
import pandas as pd
pd.__version__

'1.4.2'

In [6]:
import matplotlib.pyplot as plt

In [9]:
albums = pd.read_csv('data/albums.csv')
bands = pd.read_csv('data/bands.csv')
reviews = pd.read_csv('data/reviews.csv')

In [10]:
albums

,id,band,title,year
0,1,1,Taste the Hate,2014
1,2,3,The Bringer of Death,2014
2,3,4,Deception| Calculate What You Can't See,2005
3,4,7,Inti Huacay,1998
4,5,7,Génesis & horror,2002
...,...,...,...,...
28064,28065,37723,White Noise Paranormal,2010
28065,28066,37723,Anomalies of the Forest,2011
28066,28067,37723,Mother Earth Pantheon,2012
28067,28068,37723,Paganwinds,2014


In [36]:
df = albums.merge(bands, 
                  left_on='band', 
                  right_on='id', 
                  suffixes=(None,'_r')).drop(['band', 'id_r'], 
                                             axis=1).set_index('id')
df

,title,year,name,country,status,formed_in,genre,theme,active
id,,,,,,,,,
1,Taste the Hate,2014,('M') Inc.,United States,Unknown,2009.0,Death Metal,NaN,2009-?
2,The Bringer of Death,2014,.F.O.A.D.,France,Active,2009.0,Death Metal,Life and Death,2009-present
3,Deception| Calculate What You Can't See,2005,100 Suns,United States,Active,2004.0,Death Metal,NaN,2004-present
4,Inti Huacay,1998,1917,Argentina,Active,1994.0,Death Metal,Dark Philosophical Poetry| Art| Religion| Psyc...,1994-present
5,Génesis & horror,2002,1917,Argentina,Active,1994.0,Death Metal,Dark Philosophical Poetry| Art| Religion| Psyc...,1994-present
...,...,...,...,...,...,...,...,...,...
28065,White Noise Paranormal,2010,Frozen Mist,United States,Active,2004.0,Gothic Metal (2004-2006)| Melodic Death/Gothic...,Darkness| Despair| Death| Horror| Pagan Ancestry,2004-2014| 2014-2016| 2016-present
28066,Anomalies of the Forest,2011,Frozen Mist,United States,Active,2004.0,Gothic Metal (2004-2006)| Melodic Death/Gothic...,Darkness| Despair| Death| Horror| Pagan Ancestry,2004-2014| 2014-2016| 2016-present
28067,Mother Earth Pantheon,2012,Frozen Mist,United States,Active,2004.0,Gothic Metal (2004-2006)| Melodic Death/Gothic...,Darkness| Despair| Death| Horror| Pagan Ancestry,2004-2014| 2014-2016| 2016-present


In [37]:
reviews[['album','score']] #late

,album,score
0,4,0.88
1,5,0.96
2,8,0.83
3,45,0.80
4,54,0.92
...,...,...
21505,28061,0.89
21506,28065,0.80
21507,28065,0.80
21508,28066,0.80
